In [ ]:
#  Installing all dependencies
!pip install langchain langchain-community transformers sentence-transformers faiss-cpu langchain-text-splitters

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7

In [ ]:
# Step 01: Chunk Text and remove newlines
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

loader = TextLoader("blogs/blog1.txt")

docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=0, separators=["\n","\n\n"," ",""])
splitted_docs = splitter.split_documents(docs)
splitted_docs


[Document(metadata={'source': 'blog1.txt'}, page_content='The Future of Artificial Intelligence: Trends and Predictions'),
 Document(metadata={'source': 'blog1.txt'}, page_content='Artificial Intelligence (AI) is rapidly transforming the way we live and work. From healthcare to transportation, AI applications are making processes more efficient and accurate. One major trend is'),
 Document(metadata={'source': 'blog1.txt'}, page_content='the integration of AI into everyday consumer devices, making them smarter and more responsive.'),
 Document(metadata={'source': 'blog1.txt'}, page_content='In healthcare, AI-powered diagnostics can analyze medical images faster than human experts, enabling earlier detection of diseases such as cancer. In the transportation sector, self-driving'),
 Document(metadata={'source': 'blog1.txt'}, page_content='technology is leveraging AI to improve safety and reduce human error.'),
 Document(metadata={'source': 'blog1.txt'}, page_content='Another key developme

In [ ]:
# Step 02: Text Embedding and Vector Store
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(splitted_docs, embeddings)

vector_store.save_local("blogs/", "faiss-index")



In [ ]:
# Step 03: FAISS Retreiver with langchain
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate

db = FAISS.load_local(folder_path="blogs/",index_name="faiss-index", embeddings=embeddings, allow_dangerous_deserialization=True)
retriever = db.as_retriever()

pipeline = pipeline("text-generation", model="gpt2", max_new_tokens=200,
    temperature=0.7,
    pad_token_id=50256,
    device=0)

prompt_template = """Answer the following question from given context below appropriately and concisely, If you don't know just say I don't know
context:
{context}

question:
{question}

answer:
""".strip()


prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

llm = HuggingFacePipeline(pipeline=pipeline)



qa = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, combine_docs_chain_kwargs={"prompt": prompt})


query = "Explain me AI in one or two sentences"
chat_history = []
result = qa({"question": query, "chat_history": chat_history}, return_only_outputs=True)

print(result["answer"])













Device set to use cpu


Answer the following question from given context below appropriately and concisely, If you don't know just say I don't know
context:
the integration of AI into everyday consumer devices, making them smarter and more responsive.

Artificial Intelligence (AI) is rapidly transforming the way we live and work. From healthcare to transportation, AI applications are making processes more efficient and accurate. One major trend is

technology is leveraging AI to improve safety and reduce human error.

The coming decade will likely see AI becoming even more integrated into our lives, driving innovation while challenging us to rethink how we work, communicate, and make decisions.

question:
Explain me AI in one or two sentences

answer:

The future of AI is a huge question. For the moment

AI is the only way to solve it.

The next generation is coming

AI is the future of the human race.

The future of human civilization is in the next few years and the next AI technology will be able to take i